In [2]:
#data collec
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

data=gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\mohan\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [3]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


#load the dataset
with open('hamlet.txt','r') as file:
    text=file.read().lower()

#tokenize the data (convert the data into numbers/vectorize)
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])   
total_words=len(tokenizer.word_index)+1
total_words  


AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
total_words

4818

In [ ]:
#create input sequences
input_sequences=[]
for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequence=token_list[:i+1]
        input_sequences.append(n_gram_sequence) 

In [ ]:
max_sequence_length=max([len(x) for x in input_sequences])
max_sequence_length

14

In [ ]:
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_length,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [ ]:
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]    

In [ ]:
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout

# #define the model
# model=Sequential()
# model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_length-1))
# model.add(LSTM(150,return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(100))
# model.add(Dense(total_words,activation='softmax'))

# model.build(input_shape=(None, max_sequence_length))


# #compile the model
# model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.summary()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Load tokenizer to get total_words and sequence length
import pickle

with open("tokenizer.pickle", "rb") as f:
    tokenizer = pickle.load(f)

total_words = len(tokenizer.word_index) + 1
max_sequence_length = 15  # Change this if your training used a different sequence length

# Define the model architecture exactly as trained
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_length - 1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

# Load weights from the original .h5
model.load_weights("next_word_lstm.h5")

# Save in updated format
model.save("next_word_lstm_converted.h5")
print("✅ Converted model saved as next_word_lstm_converted.h5")


ModuleNotFoundError: No module named 'numpy'

In [ ]:
#train the model
history=model.fit(x_train,y_train,epochs=50,validation_data=(x_test,y_test),verbose=1,callback)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 33s 36ms/step - accuracy: 0.0316 - loss: 7.1385 - val_accuracy: 0.0313 - val_loss: 6.7771
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.0374 - loss: 6.4439 - val_accuracy: 0.0416 - val_loss: 6.8605
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.0449 - loss: 6.3197 - val_accuracy: 0.0445 - val_loss: 6.9101
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 20s 30ms/step - accuracy: 0.0482 - loss: 6.1713 - val_accuracy: 0.0497 - val_loss: 6.9398
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.0559 - loss: 6.0353 - val_accuracy: 0.0540 - val_loss: 6.9612
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 22ms/step - accuracy: 0.0662 - loss: 5.8570 - val_accuracy: 0.0552 - val_loss: 7.0054
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 29s 46ms/step - accuracy: 0.0734 - loss: 5.7352 - val_accuracy: 0.0595 - val_loss: 7.0371
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 19s 29ms/step - accuracy: 0.0802 - loss: 5.5809 - 

In [ ]:
def predict_next_word(model,tokenizer,text,max_sequence_length):
    token_list=tokenizer.texts_to_sequences([text])[0]

    if len(token_list)>=max_sequence_length:
        token_list=token_list[-(max_sequence_length-1):]
    token_list=pad_sequences([token_list],maxlen=max_sequence_length-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word 
    return None
        

In [ ]:
input_text='To his vnmastred '
print(f"Input Text: {input_text}")
max_sequence_length=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_length)
print(next_word)

Input Text: To his vnmastred 
importunity


In [ ]:
model.save('next_word_lstm.h5')

import pickle
with open('tokenizer.pickle','wb') as file:
    pickle.dump(tokenizer,file,protocol=pickle.HIGHEST_PROTOCOL)